# CIC Filter Notes
A Cascode Integrator Comb (CIC) filter is a special class of FIR filter often used for decimation, and interpolation tasks. The CIC filter is specially desined so that all its coefficients are 1 or 0, which greatly improves the computation time by removing the need for any multiplicaions int he DSP. 

I am interested in using this type of filter for decimation of a PDM signal form a MEMs Microphone. So I am focusing on decimation in this notebook.

## General Resources 
- https://en.wikipedia.org/wiki/Cascaded_integrator%E2%80%93comb_filter
- http://dspguru.com/files/cic.pdf
- https://wirelesspi.com/cascaded-integrator-comb-cic-filters-a-staircase-of-dsp/

## DSP Review
Here I just have some some basic DSP review to help remind myself some of the core concepts required to understand the CIC filter. 

### Discrete Time Signals vs Systems
Remeber that in general a FIR filter (or any discrete filter) can be viewd at a high level a *system*. The system transforms some input *signal* to some new output *signal*. It is common to denote a discrete time system with the letter H:
$$
x[n] \rightarrow \boxed{H} \rightarrow y[n]
$$

### Impulse Respone
Any system can be fully described by it time domain impulse respone, $h[n]$, defined as the systems output to a unit impule input signal, $x[n]=\delta[n]$

### Z-transform
To analyze the CIC filter first remember the formula for the Z-Transfom of the *signal* x:
$$
\mathbb{Z(x[n])} = X(z) = \sum_{n=-\infty}^{\infty} x[n] z^{-n}
$$

#### Time Shifting Property of Z-Transfrom
$$
\mathbb{Z}(x[n-k]) = z^{-k}X(z)
$$

## CIC Filter Thoery
The basic approach to a decemation CIC filter is 3 steps
1. Integrate
2. Downsample
1. Comb Filter

$$
x[n] \rightarrow \fbox{I} \rightarrow \fbox{\downarrow} \rightarrow \fbox{C} \rightarrow[n]
$$

The CIC interpolator follows a similar, but reveresed structure:
1. Comb Filter
2. Upsample
3. Integrate

$$
x[n] \rightarrow \fbox{C} \rightarrow \fbox{\uparrow} \rightarrow \fbox{I} \rightarrow[n]
$$



### Integrator: A simple IIR Filter

#### Basic Equation
A digital integrator is defined by the following equation:
$$
y[n] = y[n-1] + x[n]
$$
*Note this is an IIR system becasue the output $y[n]$ depends on past outputs $y[n-1]$.*

Lets look at what this equation is actually saying, by seeing what happens as more and more samples are collected. 

Assume we are sampling some analog signal and that is our input $x[n]$

| $n$ | $y[n]$ | $h[n]$ assume ($y[-1]=0$) |
| :-- | :-- | :-- |
| $n=0$ | $y[0] = y[-1] + x[0]$ | $h[0] = 1$ |
| $n=1$ | $y[1] = y[0] + x[1] = y[-1] + x[0] + x[1]$ | $h[1] = 1 + 0$ |
| $n=2$ | $y[2] = y[1] + x[2] = y[-1] + x[0] + x[1] + x[2]$ | $h[2] = 1+ 0 + 0$ |
| $n=3$ | $y[3] = y[2] + x[3] = y[-1] + x[0] + x[1] + x[2] + x[3]$ | $h[3] = 1 + 0 + 0 + 0$ |
| $n=k$ | $y[k] = y[k-1] + x[k] = y[-1] + x[0] + x[1] + ... x[k-1] + x[k]$ | $h[k] = u[n]$ |

Looking at it like this the pattern becomes clear. The integrater is simply a ruinning sum of each sample of the input signal. So when we implement this in code we simply want to add each new sample to some variable that holds the total sum. If dont really know what $y[-1]$, it will present itself as some offset, but going forward I will assume $y[-1]=0$

For this reason the Integrator is often reffered to as an *accumulator*.

#### Transfer Funtion
We can find the transfer function of the Integrator as follows:

First take the Z-transform of our system:
$$
Y(z) = \mathbb{Z}(y[n-1]) + X(z)
$$
We can use the time-shifing property of the Z-transform to get the follwing
$$
Y(z) = z^{-1}Y(z) + X(z)
$$
Now we can easily solve for the transfer function $H(z)$
$$
Y(Z) \left[1 - z^{-1}\right] = X(x)
$$
$$
\boxed{H(z) = \frac{Y(z)}{X(z)} = \frac{1}{1 - z^{-1}}}
$$

This system has no zeros becasue there is no value of $z$ that make $H(z) = 0$

We can quickly solve for the systems poles by finding where $|H(z)| = \infty$ (aka the demoninator is 0)
$$
0 = 1-\frac{1}{z}
$$
$$
1 = \frac{1}{z}
$$
$$
z=1
$$
So we can see the system had a single pole at $z=1$

### Comb Filter: A simple FIR Filter
The Comb filter after downsampling can be shon to hae the following system equation:
$$
y[n] = x[n] - x[n-M]
$$
Which corresponds to:
$$
Y(z) = X(z) - z^{-M}X(z)
$$
and 
$$
\boxed{H(z) = 1-z^{-M}}
$$

This Filter has no poles and 1 zero at $z=1$

### Putting it all together 

The CIC filter passed the input through $N$ integrateor stages, then through a downsampler with rate of $R$, and finally through $N$ Comb stages

We saw that:
$$
H_I(z) = \frac{1}{1-z^-1}
$$
and 
$$
H_C(z) = 1-z^{-RM}
$$

In LTI systems we know that convolution = multiplication so for $N$ integrators we get:
$$
H_1(z) = H_I^N(z) = \frac{1}{(1-z^-1)^N}
$$
and for $N$ Comb filters we have:
$$
H_2(z) = H_C^N(z) = (1-z^{-RM})^N
$$

Cobmining everython gives:
$$
H(z) = H_1(z) \times H_2(z) = \frac
    {\left( 1-z^{-RM} \right)^N}
    {\left( 1-z^{-1} \right)^N}
$$
This can be shown to simplify down to:
$$
H(z) = \left( \sum_{k=0}^{RM-1}z^{-k}\right)^N
$$

# Simulation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
Ntaps = 15
hn = np.ones(15)

In [19]:
def integrate(x:np.ndarray)->np.ndarray:
    h = np.ones(np.size(x))
    y = np.convolve(h, x, 'same')
    return y
def downsample(x:np.ndarray, R:int)->np.ndarray:
    y = x[0::R]
    return y
def comb():
    h = np.zeros(np.

[1. 3. 6. 6.]
